In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
import gensim

model = gensim.models.word2vec.Word2Vec.load("model/word2vec")

In [2]:
df = pd.read_excel('data_processed.xlsx')[['身份','情绪','发帖内容']]
df = df[df['身份']==1].dropna()
reviews = []
keys=[]
for sentiment in range(-1,2):
    if sentiment == -1:
        df_i = df[df['情绪']==sentiment].reset_index().drop(columns='index')
        for i in range(len(df_i['情绪'])):
            reviews.append(df_i['发帖内容'][i])
            keys.append('neg'+'_'+str(i))
    elif sentiment == 0:
        df_i = df[df['情绪']==sentiment].reset_index().drop(columns='index')
        for i in range(len(df_i['情绪'])):
            keys.append('neu'+'_'+str(i))
            reviews.append(df_i['发帖内容'][i])
    else: 
        df_i = df[df['情绪']==sentiment].reset_index().drop(columns='index')
        for i in range(len(df_i['情绪'])):
            keys.append('pos'+'_'+str(i))
            reviews.append(df_i['发帖内容'][i])
docDict = dict(zip(keys,reviews))
docDict

{'neg_0': '老师，棕榈晚上高开后是反手做空吗？',
 'neg_1': '黄金1455支撑，多单进场分析黄金目前1458上方，上方1463压制价格下跌，大趋势上看空。但下方支撑强势，若要跌破，需进一步确认，1456轻仓多，多单较稳健。',
 'neg_2': '白银空头大幅增仓，库存大幅增加，逢高沽空白银空头大幅增仓，库存大幅增加，逢高沽空',
 'neg_3': '白银逢高沽空',
 'neg_4': '大空头持续增仓',
 'neg_5': '白银空头大幅增仓，库存大幅增加',
 'neg_6': '白银下午跳水，今晚会一泻千里白银下午跳水，今晚会一泻千里',
 'neg_7': '《无尽空虚，做多不急》我89后。但我还是喜欢张国荣，张学友、beyond等，很多朋友嘲笑我，说我out，不懂时尚，我不说话，喜欢谁是我的自由，我不会因为别人的三言两语而改变我对他们的崇敬。他们创造的无数经典，是现在这些所谓的流量明星和歌手难以超越的。这首beyond的《无尽空虚》很适合在爆赚之后，当天晚上去听，最好喝的微醺，带上你的耳机，声音调大点，一个人在房间，关上门，独自享受那种痛并快乐的感觉。左手拿着手机看着账户上的几万美金浮盈，右手拿着82年的科罗娜，那种感觉美的无尽藏。或许那时你会喜极而泣，因为你等的太久了，也因为你亏得太多了，那个时候你脑海中或许会浮现各种往事所经历的场景。脑海浮现，曾经期待的爱，怎么一生都碰不到，想起曾经离自己而去的女朋友和那种悲伤分手的场面。就像如果你是一个演员，你等了10年终于获得《金像奖》影帝、影后一样。我相信一切的努力和隐忍，都将成为你未来2-3年快速崛起的必然基石。交易并不是你想的那样美好，预期想的复杂难耐，不如简约、大方却又不失细腻来得更加实在。.........近日，由于那啥乐观情绪回升，使得美股再次刷新历史新高，真是刷新大众对于牛市的认知，十年牛市，一直在超越，从未被模仿。黄金也再次快速跌破1450，而后由于美国相关数据良莠不齐，造成V型反转，不过没有多大影响，重头戏在今天，再次强调今天是关键。为什么关键?前面文章也提过了。21:30美国第三季度实际GDP年化季率、美国至11月23日当周初请失业金人数。21:30美国10月耐用品订单月率、美国第三季度核心PCE物价指数年化季率。22:45美国11月芝加哥PMI。23:00美国10

In [3]:
def wordVecs(wordList):
    vecList = []
    for word in wordList:  
        try:
            vecList.append(model[word])
        except KeyError:
            continue
    return vecList

def decVecs(docDict):
    global trainSet, acLabel
    trainSet = []
    acLabel = []
    for key in docDict.keys():
        if type(docDict[key]) == str:
            resultList = wordVecs(docDict[key])
            # 所有向量的均值向量都用来表示文本
            # print(resultList)
            if resultList is not None and len(resultList) != 0:
                resultArray = sum(np.array(resultList))/len(resultList)
                trainSet.append(resultArray)
                if key[:3] == 'pos':
                    acLabel.append(1)
                elif key[:3] == 'neg':
                    acLabel.append(-1)
                else: acLabel.append(0)
                
decVecs(docDict)

/Users/alice/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [4]:
# 训练
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler  
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

X_train,X_test,y_train,y_test = train_test_split(trainSet,acLabel,test_size=0.1,random_state=0)
ss = StandardScaler()                    
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)
 
lsvc = LinearSVC(C=0.01)                      
# 初始化现行假设的支持向量机分类器 LinearSVC
lsvc.fit(X_train,y_train)                
# 模型训练
y_predict = lsvc.predict(X_test)

#生成结果报告
print ('The Accuracy of Linear SVC is %f'%lsvc.score(X_test,y_test))   
# 使用自带的模型评估函数进行准确性评测
print(classification_report(y_test,y_predict))


The Accuracy of Linear SVC is 0.693878
              precision    recall  f1-score   support

          -1       0.68      0.61      0.64        28
           0       0.69      0.71      0.70        35
           1       0.70      0.74      0.72        35

    accuracy                           0.69        98
   macro avg       0.69      0.69      0.69        98
weighted avg       0.69      0.69      0.69        98



In [5]:
from sklearn.ensemble import RandomForestClassifier

X_train,X_test,y_train,y_test = train_test_split(trainSet,acLabel,test_size=0.1,random_state=0)
ss = StandardScaler()                    
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

rfc = RandomForestClassifier(n_estimators=2000)
rfc.fit(X_train,y_train)
rfc_pred = rfc.predict(X_test)
print('The accuracy of RandomForrest Classifier is', metrics.accuracy_score(rfc_pred, y_test))

matrix_train = metrics.confusion_matrix(y_train,rfc.predict(X_train))
print('Confusion matrix on training set：\n',matrix_train)
report_train = metrics.classification_report(y_test,rfc.predict(X_test))
print('Classification on training set：\n',report_train)


The accuracy of RandomForrest Classifier is 0.6938775510204082
Confusion matrix on training set：
 [[275   0   0]
 [  0 333   1]
 [  0   0 268]]
Classification on training set：
               precision    recall  f1-score   support

          -1       0.76      0.57      0.65        28
           0       0.61      0.89      0.72        35
           1       0.81      0.60      0.69        35

    accuracy                           0.69        98
   macro avg       0.73      0.69      0.69        98
weighted avg       0.72      0.69      0.69        98



In [6]:
from sklearn.neural_network import MLPClassifier

X_train,X_test,y_train,y_test = train_test_split(trainSet,acLabel,test_size=0.1,random_state=0)
ss = StandardScaler()                    
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

mlp = MLPClassifier(hidden_layer_sizes=(200,50,10),alpha=0.1,max_iter=500,solver='lbfgs',verbose=True,warm_start=True) 
mlp.fit(X_train,y_train)
mlp_pred = mlp.predict(X_test)
print('The loss on training set is',mlp.loss_)
print('The accuracy of Multi-layer Perceptron Classifier is', metrics.accuracy_score(mlp_pred, y_test))

matrix_train = metrics.confusion_matrix(y_train,mlp.predict(X_train))
print('Confusion matrix on training set：\n',matrix_train)
report_train = metrics.classification_report(y_test,mlp.predict(X_test))
print('Classification on training set：\n',report_train)


The loss on training set is 0.006994938957283758
The accuracy of Multi-layer Perceptron Classifier is 0.6836734693877551
Confusion matrix on training set：
 [[275   0   0]
 [  0 333   1]
 [  0   0 268]]
Classification on training set：
               precision    recall  f1-score   support

          -1       0.66      0.68      0.67        28
           0       0.69      0.77      0.73        35
           1       0.70      0.60      0.65        35

    accuracy                           0.68        98
   macro avg       0.68      0.68      0.68        98
weighted avg       0.68      0.68      0.68        98



In [26]:
from sklearn.linear_model import LogisticRegression

X_train,X_test,y_train,y_test = train_test_split(trainSet,acLabel,test_size=0.1,random_state=7)
ss = StandardScaler()                    
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

lr = LogisticRegression(C=0.004,solver='lbfgs',verbose=True,warm_start=True,multi_class='auto')
lr.fit(X_train,y_train)
lr_pred = lr.predict(X_test)
print('The accuracy of Logistic Regression is', lr.score(X_test,y_test))

matrix_train = metrics.confusion_matrix(y_train,lr.predict(X_train))
print('Confusion matrix on training set：\n',matrix_train)
report_train = metrics.classification_report(y_test,lr.predict(X_test))
print('Classification on training set：\n',report_train)

The accuracy of Logistic Regression is 0.6020408163265306
Confusion matrix on training set：
 [[158  70  41]
 [ 48 249  40]
 [ 32  58 181]]
Classification on training set：
               precision    recall  f1-score   support

          -1       0.66      0.56      0.60        34
           0       0.50      0.72      0.59        32
           1       0.74      0.53      0.62        32

    accuracy                           0.60        98
   macro avg       0.63      0.60      0.60        98
weighted avg       0.63      0.60      0.60        98



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


In [44]:
scores = [lsvc.score(X_test,y_test),
          rfc.score(X_test,y_test), 
          mlp.score(X_test,y_test), 
          lr.score(X_test,y_test)]
score_df = pd.DataFrame({
    'Model' : ['Linear SVC','RandomForest','MLP','Logistic Regression'],
    'Scores' : scores
})
score_df.set_index('Model')
score_df

,Model,Scores
0,Linear SVC,0.730769
1,RandomForest,0.769231
2,MLP,0.846154
3,Logistic Regression,0.807692


In [7]:
file = pd.read_excel('identity_labeled.xlsx').drop(columns='Unnamed: 0')
file = file[file['身份'] == 1].dropna()
test = file[['身份','发帖内容']]

testSet = []
for i in test['发帖内容']:
    word_vec = wordVecs(i)
    if word_vec is not None and len(word_vec) != 0:
        resultArray = sum(np.array(word_vec))/len(word_vec)
        testSet.append(resultArray)
        
testX = ss.fit_transform(testSet)
y_pred = rfc.predict(testX)

file['情绪'] = y_pred
file.to_excel('sentiment_labeled.xlsx')

/Users/alice/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
